# Tutorial part 2
## Useful packages in the Brian ecosystem

### brian2tools: tools for visualization and import/export

https://brian2tools.readthedocs.io/

In [ ]:
from brian2 import *
prefs.codegen.target = 'numpy'
%matplotlib notebook

In [ ]:
# I/F curve example for HH model
start_scope()

N = 1000
duration = 1*second
area = 20000*umetre**2
Cm = 1*ufarad*cm**-2 * area
gl = 5e-5*siemens*cm**-2 * area
El = -65*mV
EK = -90*mV
ENa = 50*mV
g_na = 100*msiemens*cm**-2 * area
g_kd = 30*msiemens*cm**-2 * area
VT = -63*mV

# The model
eqs = Equations('''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*4*mV/exprel((13.*mV-v+VT)/(4*mV))/ms*(1-m)-0.28*(mV**-1)*5*mV/exprel((v-VT-40.*mV)/(5*mV))/ms*m : 1
dn/dt = 0.032*(mV**-1)*5*mV/exprel((15.*mV-v+VT)/(5*mV))/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
I : amp
''')
# Threshold and refractoriness are only used for spike counting
group = NeuronGroup(N, eqs,
                    threshold='v > -40*mV',
                    refractory='v > -40*mV',
                    method='exponential_euler')
group.v = El
group.I = '0.7*nA * i / N'

monitor = SpikeMonitor(group)
v_mon = StateMonitor(group, 'v', record=True)

run(duration, report='text')

In [ ]:
from brian2tools import brian_plot

In [ ]:
plt.figure()
brian_plot(monitor);

In [ ]:
plt.figure()
brian_plot(v_mon[50]);

In [ ]:
start_scope()
G = NeuronGroup(10, '')
# Random connectivity + condition
S = Synapses(G, G)
S.connect('i != j', p=0.5)

In [ ]:
plt.figure()
brian_plot(S);

In [ ]:
start_scope()
G = NeuronGroup(10000, '')
S = Synapses(G, G)
S.connect('i != j', p='exp(-abs(i - j)/250)')

In [ ]:
plt.figure()
brian_plot(S);

In [ ]:
# NeuroML export

import brian2tools.nmlexport

start_scope()
set_device('neuroml2', filename="nml2model.xml")

n = 100
duration = 1*second
tau = 10*ms

eqs = '''
dv/dt = (v0 - v) / tau : volt (unless refractory)
v0 : volt
'''
group = NeuronGroup(n, eqs, threshold='v > 10*mV', reset='v = 0*mV',
                    refractory=5*ms, method='exact')
group.v = 0*mV
group.v0 = '20*mV * i / (N-1)'

rec_idx = [2, 63]
statemonitor = StateMonitor(group, 'v', record=rec_idx)
spikemonitor = SpikeMonitor(group, record=rec_idx)

run(duration)

In [ ]:
%cat nml2model.xml

In [ ]:
# Markdown export

import brian2tools.mdexport

start_scope()
set_device('markdown', filename="network")

n = 100
duration = 1*second
tau = 10*ms

eqs = '''
dv/dt = (v0 - v) / tau : volt (unless refractory)
v0 : volt
'''
group = NeuronGroup(n, eqs, threshold='v > 10*mV', reset='v = 0*mV',
                    refractory=5*ms, method='exact')
group.v = 0*mV
group.v0 = '20*mV * i / (N-1)'

rec_idx = [2, 63]
statemonitor = StateMonitor(group, 'v', record=rec_idx)
spikemonitor = SpikeMonitor(group, record=rec_idx)

run(duration)

In [ ]:
%cat network.md

### brian2modelfitting

Automated model fitting and simulation-based inference

In [ ]:
# Not installed on OSBv2
%pip install brian2modelfitting
%pip install sbi

In [ ]:
from brian2 import *
from brian2modelfitting import *
set_device('runtime')

In [ ]:
import pandas as pd
inp_trace = pd.read_csv('input_traces_hh.csv', index_col=0).to_numpy()*amp
out_trace = pd.read_csv('output_traces_hh.csv', index_col=0).to_numpy()*mV
dt = 0.01*ms
times = np.arange(inp_trace.shape[1])*dt

In [ ]:
fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(8, 4), constrained_layout=True)
ax_left.plot(times/ms, inp_trace.T/nA)
ax_right.plot(times/ms, out_trace.T/mV)
ax_left.set(xlabel='time (ms)', ylabel='I (na)')
ax_right.set(xlabel='time (ms)', ylabel='v (mV)');

In [ ]:
area = 20000*umetre**2
Cm=1*ufarad*cm**-2 * area
El=-65*mV
EK=-90*mV
ENa=50*mV
VT=-63*mV

# The unknown parameter are defined as constant parameters in the model:
model = '''
dv/dt = (gl*(El-v) - g_na*(m*m*m)*h*(v-ENa) - g_kd*(n*n*n*n)*(v-EK) + I)/Cm : volt
dm/dt = 0.32*(mV**-1)*(13.*mV-v+VT)/
  (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms*(1-m)-0.28*(mV**-1)*(v-VT-40.*mV)/
  (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms*m : 1
dn/dt = 0.032*(mV**-1)*(15.*mV-v+VT)/
  (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms*(1.-n)-.5*exp((10.*mV-v+VT)/(40.*mV))/ms*n : 1
dh/dt = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms*(1.-h)-4./(1+exp((40.*mV-v+VT)/(5.*mV)))/ms*h : 1
g_na : siemens (constant)
g_kd : siemens (constant)
gl   : siemens (constant)
'''

In [ ]:
opt = NevergradOptimizer()
metric = MSEMetric()

In [ ]:
fitter = TraceFitter(model=model,
                     input_var='I',
                     output_var='v',
                     input=inp_trace,
                     output=out_trace,
                     dt=0.01*ms,
                     n_samples=100,
                     method='exponential_euler',
                     param_init={'v': -65*mV})

In [ ]:
# Start fit, provide ranges for parameters
del _exit_code  # ignore, this fixes an ugly bug with brian2modelfitting and the notebook
area = 20000*umetre**2
res, error = fitter.fit(n_rounds=10,
                        optimizer=opt,
                        metric=metric,
                        gl=[2*psiemens, 200*nsiemens],
                        g_na=[200*nsiemens, 0.4*msiemens],
                        g_kd=[200*nsiemens, 200*usiemens])

In [ ]:
traces = fitter.generate_traces()

In [ ]:
plt.figure()
plt.plot(times/ms, out_trace[3]/mV, label='data')
plt.plot(times/ms, traces[3]/mV, label='fit')
plt.legend()
plt.xlabel('time (ms)')
plt.ylabel('v (mV)');

In [ ]:
refined_params, result_info = fitter.refine(calc_gradient=True)

In [ ]:
traces = fitter.generate_traces(params=refined_params)

In [ ]:
plt.figure()
plt.plot(times/ms, out_trace[3]/mV, label='data')
plt.plot(times/ms, traces[3]/mV, label='fit')
plt.legend()
plt.xlabel('time (ms)')
plt.ylabel('v (mV)');

Also support for *simulation-based inference*:

In [ ]:
%%html
<iframe width=900 height=600 src="https://www.mackelab.org/sbi/"></iframe>

## Code generation for GPU

In [ ]:
start_scope()
set_device('cpp_standalone')
# # GPU via GeNN simulator
# import brian2genn
# set_device('genn')

# # GPU via CUDA code generation
# import brian2cuda
# set_device('cuda_standalone')

# Parameters
area = 20000*umetre**2
Cm = (1*ufarad*cm**-2) * area
gl = (5e-5*siemens*cm**-2) * area

El = -60*mV
EK = -90*mV
ENa = 50*mV
g_na = (100*msiemens*cm**-2) * area
g_kd = (30*msiemens*cm**-2) * area
VT = -63*mV
# Time constants
taue = 5*ms
taui = 10*ms
# Reversal potentials
Ee = 0*mV
Ei = -80*mV
we = 6*psiemens  # excitatory synaptic weight
wi = 67*psiemens  # inhibitory synaptic weight

# The model
eqs = Equations('''
dv/dt = (gl*(El-v)+ge*(Ee-v)+gi*(Ei-v)-
         g_na*(m*m*m)*h*(v-ENa)-
         g_kd*(n*n*n*n)*(v-EK))/Cm : volt
dm/dt = alpha_m*(1-m)-beta_m*m : 1
dn/dt = alpha_n*(1-n)-beta_n*n : 1
dh/dt = alpha_h*(1-h)-beta_h*h : 1
dge/dt = -ge*(1./taue) : siemens
dgi/dt = -gi*(1./taui) : siemens
alpha_m = 0.32*(mV**-1)*(13.*mV-v+VT)/
         (exp((13.*mV-v+VT)/(4.*mV))-1.)/ms : Hz
beta_m = 0.28*(mV**-1.)*(v-VT-40.*mV)/
        (exp((v-VT-40.*mV)/(5.*mV))-1.)/ms : Hz
alpha_h = 0.128*exp((17.*mV-v+VT)/(18.*mV))/ms : Hz
beta_h = 4./(1.+exp((40.*mV-v+VT)/(5.*mV)))/ms : Hz
alpha_n = 0.032*(mV**-1.)*(15.*mV-v+VT)/
         (exp((15.*mV-v+VT)/(5.*mV))-1.)/ms : Hz
beta_n = .5*exp((10.*mV-v+VT)/(40.*mV))/ms : Hz
''')

P = NeuronGroup(40000, model=eqs, threshold='v>-20*mV', refractory=3*ms,
                method='exponential_euler')
Pe = P[:32000]
Pi = P[32000:]
Ce = Synapses(Pe, P, on_pre='ge+=we')
Ci = Synapses(Pi, P, on_pre='gi+=wi')
Ce.connect(p=0.02)
Ci.connect(p=0.02)

# Initialization
P.v = 'El + (randn() * 5 - 5)*mV'
P.ge = '(randn() * 1.5 + 4) * 10.*nS'
P.gi = '(randn() * 12 + 20) * 10.*nS'

# Record a few traces
trace = StateMonitor(P, 'v', record=[1, 10, 100])
run(1 * second, report='text')

In [ ]:
plt.figure()
plt.plot(trace.t/ms, trace[1].v/mV)
plt.plot(trace.t/ms, trace[10].v/mV)
plt.plot(trace.t/ms, trace[100].v/mV)
plt.xlabel('t (ms)')
plt.ylabel('v (mV)');